In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time

source = requests.get("https://timesofindia.indiatimes.com/")
soup = BeautifulSoup(source.content,'html')
data = []
links = []

def remove_duplicates(l):
    for item in l:
        match = re.search("(?P<url>https?://[^\s]+)",item)
        if match is not None:
            links.append((match.group("url")))

for links in soup.find_all('a',href=True):
    data.append(str(links.get('href')))
flag=True
remove_duplicates(data)

while flag:
    try:
        for link in links:
            for j in soup.find_all('a',href=True):
                temp=[]
                source =requests.get(link)
                soup=BeautifulSoup(source.content,'html')
                temp.append(str(j.get('href')))
                remove_duplicates(temp)
                
                if len(links)>162:
                    break
            if len(links)>162:
                break
        if len(links)>162:
            break
    except Exception as e:
        print(e)
        if len(links)>162:
            break

for urls in links:
    print(urls)

Invalid URL 'Times Syndication Service': No schema supplied. Perhaps you meant http://Times Syndication Service?
Times Syndication Service
https://www.timespoints.com/products?utm_source=TOI&utm_medium=timespoint&utm_campaign=daily_checkin
https://jsso.indiatimes.com/sso/identity/profile/edit?channel=toi
https://mytimes.indiatimes.com/?channel=toi
https://www.timespoints.com/#/products?utm_source=TOI&utm_medium=Header_Dropdown&utm_campaign=TOI_HeaderDropdown
https://twitter.com/timesofindia
https://www.facebook.com/TimesofIndia
https://timesofindia.indiatimes.com/rss.cms
https://www.youtube.com/user/TimesOfIndiaChannel
https://recipes.timesofindia.com/videos/videolist/52254947.cms
https://navbharattimes.indiatimes.com/video/videolist/4901865.cms
http://photogallery.indiatimes.com/news/photodhamal/articlelist/14582101.cms
https://www.facebook.com/toiworldnews
http://photogallery.indiatimes.com/News/Business/articlelist/15466935.cms
https://twitter.com/TOIBusinessNews
https://www.faceboo

In [ ]:
from goose3 import Goose
g=Goose()
url_content= []
for url in urls:
    page = g.extract(url='https://timesofindia.indiatimes.com/')
    soup=BeautifulSoup(page.raw_html,"html")
    links=soup.find_all('a')
    for l in links:
        link=l.get('href')
        if link and link.find('Technology'):
            url_content.append(link)
            #print(url_content)
for url in urls:
    try: 
        articles=g.extract(url=url)
        content = articles.cleaned_text
        if len(content)>200:
            title=articles.title
            print(title)
            with open('homework/goose/'+articles.title+'.txt','w') as f:
                f.write(content)
    except:
        pass

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# define the function for lemmatization
def lemma_tokenizer(text):
    # use the standard scikit-learn tokenizer first
    standard_tokenizer = CountVectorizer().build_tokenizer()
    tokens = standard_tokenizer(text)
    # then use NLTK to perform lemmatisation on each token
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemma_tokens=[]
    for token in tokens:
        if re.search('[a-zA-Z]', token):  # save those which are non-numeric
            lemma_tokens.append(lemmatizer.lemmatize(token))
    return lemma_tokens

In [ ]:
#import glob
#articles=g.extract(url="https://timesofindia.indiatimes.com/")
docs_text=[]  #store text of all documents
for url in urls:
    page = g.extract(url='https://timesofindia.indiatimes.com/')
    soup=BeautifulSoup(page.raw_html,"html")
    links=soup.find_all('a')
    for l in links:
        link=l.get('href')
        if link and link.find('article'):
            docs_text.append(link)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
#chosen n-gram of three words. It will produce phrases containing upto three words
vectorizer = TfidfVectorizer(min_df=5,stop_words='english',tokenizer=lemma_tokenizer, ngram_range=(1,3)) 
#fit the vectorizer to documents
tfidf_matrix = vectorizer.fit_transform(docs_text)

In [ ]:
tfidf_matrix

In [ ]:
while(True):
    try:
        terms = vectorizer.get_feature_names()
        feature_array = np.array(terms)
        tfidf_sorting = np.argsort(tfidf_matrix.toarray()).flatten()[::-1]
        n = 30
        top_n = feature_array[tfidf_sorting][:n]
        
    except :
        print(top_n)
        break